In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras import metrics

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
df_collab = pd.read_csv('./data/review_all_clean.csv')
df_collab.drop(columns='reviewText', axis=1, inplace=True)
df_collab.rename(columns={'overall': 'rating', 'asin': 'movie_id', 'reviewerID': 'user_id'}, inplace=True)
df_collab.head()

,rating,user_id,movie_id
0,5,A2VHSG6TZHU1OB,0001527665
1,5,A1KM9FNEJ8Q171,0001527665
2,4,A38LY2SSHVHRYB,0001527665
3,5,AHTYUW2H1276L,0001527665
4,5,A3M3HCZLXW0YLF,0001527665


In [28]:
df_content = pd.read_csv('./data/meta_all_clean.csv')
df_content.head()

,genre,description,title,starring,rank,asin,english
0,Christian Video,An early movie edition of the life of Jesus.,Where Jesus Walked VHS,Various,1108385,0005000009,1
1,Movies,"In Depression-era New England, a miserly busin...",An American Christmas Carol VHS,Various,704028,0005019281,1
2,Documentary,This documentary takes you on a journey...from...,A NATION ADRIFT A Chronicle of America's Prov...,Tom Kane,496880,0005092663,1
3,Science Fiction & Fantasy Science Fiction Anim...,This is The VHS Movie: SANTA CLAUS IS COMIN TO...,Santa Claus Is Comin' To Town VHS,Fred Astaire,347709,0307142493,1
4,Sony Pictures Home Entertainment,"Arthur, the hapless, hugely popular star of hi...",Arthur's Perfect Christmas VHS,Various,134050,0375810331,1


In [ ]:
df_collab.info()

In [ ]:
df_collab['user_id'].value_counts()

In [ ]:
df_collab['user_id'].value_counts()[df_collab['user_id'].value_counts() >= 3]

In [4]:
df_three = df_collab[df_collab['user_id'].isin(df_collab['user_id'].value_counts()[df_collab['user_id'].value_counts() >= 3].index)]
df_three.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1274639 entries, 7 to 3013830
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   rating    1274639 non-null  int64 
 1   user_id   1274639 non-null  object
 2   movie_id  1274639 non-null  object
dtypes: int64(1), object(2)
memory usage: 38.9+ MB


In [5]:
df = df_three.copy()

In [19]:
df['rating'].value_counts()

rating
5    660689
4    278907
3    159236
1     93704
2     82103
Name: count, dtype: int64

In [6]:
df['user_id'] = df['user_id'].astype('category').cat.codes
df['movie_id'] = df['movie_id'].astype('category').cat.codes

In [7]:
num_users = df['user_id'].nunique()
num_movies = df['movie_id'].nunique()

In [8]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [24]:
user_input = tf.keras.layers.Input(shape=(1,))
movie_input = tf.keras.layers.Input(shape=(1,))

user_embedding = tf.keras.layers.Embedding(input_dim=num_users, output_dim=50)(user_input)
movie_embedding = tf.keras.layers.Embedding(input_dim=num_movies, output_dim=50)(movie_input)

concat = tf.keras.layers.concatenate([user_embedding, movie_embedding])
dense = tf.keras.layers.Dense(50, activation='relu')(concat)

output = tf.keras.layers.Dense(1)(dense)

model = tf.keras.models.Model(inputs=[user_input, movie_input], outputs=output)
model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics='mse')


model.fit([train_df['user_id'], train_df['movie_id']], train_df['rating'], epochs=5)

sparse_categorical_crossentropy = model.evaluate([test_df['user_id'], test_df['movie_id']], test_df['rating'])

Epoch 1/5
 2375/31866 [=>............................] - ETA: 1:39 - loss: nan - mse: nan

KeyboardInterrupt: 

In [10]:
model.save('./models/tensor_collab_base.h5')

In [16]:
check = tf.keras.models.load_model('./models/tensor_collab_base.h5')

In [17]:
check.evaluate([test_df['user_id'], test_df['movie_id']], test_df['rating'])

7967/7967 [==============================] - 18s 2ms/step - loss: 4.8186e-07


4.818598995370849e-07

In [ ]:
with tf.device('/gpu:0'):
    model.fit([train_df['user_id'], train_df['movie_id']], train_df['rating'], epochs=10, batch_size=32)

___